## Current status of the notebook

This notebook is meant for matching of CPE URIs to CC certificates and for subsequent matching of CVEs to the respective certificates. This is achieved as follows:

- `JSONs` with CVE data are fetched from [nist.gov](https://nvd.nist.gov/vuln/data-feeds), relevant fields extracted and all data merged into single file. Functions `download_cve_data()` and `parse_all_cves()` take care of that. 
- `XML` file with [CPE records](https://nvd.nist.gov/products/cpe) is parsed to extract solely the title and CPE uri of all records. Functions ` get_cpe_uri_to_title_dict()`
- CPE records are parsed into triplets `(vendor, product name, version)` and fetched into various dictionaries for `O(1)` access.
- The `CommonCriteriaDataset` (see [GitHub repo](https://github.com/crocs-muni/sec-certs/blob/master/sec_certs/dataset.py)) is loaded as pandas dataframe
- After some pre-processing, the function `match_cpe()` is called that based on the CC-certificate triplet `(vendor, certificate name, version)` attempts to find relevant CPE field, this is done as described below

### Matching algorithm

- First, the `vendor` goes through several heuristics and candidate vendors from the CPE-record database are found
- Second, from the certificate, list of possible versions are extracted by regex matching. For instance, the string `IDOneClassIC Card : ID-One Cosmo 64 RSA v5.4 and applet IDOneClassIC v1.0 embedded on P5CT072VOP` will match possible versions `5.4` and `1.0`. Out of the cartesian product `candidate vendors x candidate versions`, list of existing pairs is found in the CPE database. 
- For each of the candidate `(vendor, version)` pairs, list of relevant CPE product names is retrieved.
- Two fuzzy string matching algorithms are run against each of the candidate `cpe item`s and the certificate name. Best score out of all candidates is counted.
- If no candidate with high-enough score was found (all `<60`), the requirement for version match is relaxed and the pair `(vendor, -)`, i.e. unknown version, is attempted. Stronger requirements are put on results produced by this branch.
- All candidates with score `>70` and text length `>5` (otherwise it's easy to get high fuzzy-match score by accident) are considered promising.
- The promising candidates are then to be manually checked by the analyst.

### TODO

There are multiple enhancmenents that can be done:

- Better parsing of product versions
- `1:n` matching, where some CC certificates have name like `McAfee Change Control and Application Control 8.3.0 with ePolicy Orchestrator 5.10.0` and CPE record exist both for `Change Control and Application Control 8.3.0` and for `ePolicy Orchestrator 5.10.0`
- Vendor and version could be stripped while doing final fuzzy matching on `(cert name, cpe item)`
- CPE titles could be used for matching instead of the `cpe item` field
- The algorithm can be tuned to prefer more general version (for which the CVE will more likely apply)


## Representativness of the acquired dataset

In the bottom part of the notebook, several plots can be drawn to compare the feature distribution of the CPE-matched subset of CC dataset with the full CC dataset. The point is to rule-out a bias in the selection.

In [1]:
import pandas as pd
import json
import re
from rapidfuzz import process, fuzz
import numpy as np
from typing import Tuple, List, Optional
import itertools
from tqdm.auto import tqdm
import ast
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import glob
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import requests
import tempfile
import sys
import zipfile

sys.path.append('./..')
import sec_certs.helpers as helpers

tqdm.pandas()
plt.style.use('seaborn')
pd.set_option("max_colwidth", 100)
pd.set_option("max_rows", 100)

replace_non_letter_non_numbers_with_space = re.compile(r"(?ui)\W")

## Functions for CVE and CPE preprocessing

In [2]:
def download_cve_data(output_dir: str, start_year=2002, end_year=2021):
    output_dir = Path(output_dir)
    if not output_dir.exists:
        output_dir.mkdir()

    base_url = 'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-'
    urls = [base_url + str(x) + '.json.zip' for x in range(start_year, end_year + 1)]

    print(f'Identified {len(urls)} CVE files to fetch from nist.gov. Downloading them into {output_dir}', flush=True)
    with tempfile.TemporaryDirectory() as tmp_dir:
        outpaths = [Path(tmp_dir) / Path(x).name.rstrip('.zip') for x in urls]
        responses = list(zip(*helpers.download_parallel(list(zip(urls, outpaths)), num_threads=8)))[1]

        for o, u, r in zip(outpaths, urls, responses):
            if r == 200:
                with zipfile.ZipFile(o, 'r') as zip_handle:
                    zip_handle.extractall(output_dir)
            else:
                print(f'Failed to download from {u}, got status code {r}')

def parse_all_cves(cve_dir: str, output_path: str) -> None:
    def get_relevant_info_from_file(input_path: Path) -> List[Dict]:
        with input_path.open('r') as handle:
            data = json.load(handle)
        cve_data = []
        for cve in data['CVE_Items']:
            cve_data.append(get_relevant_info_from_cve(cve))
        return cve_data
    
    def get_relevant_info_from_cve(cve: Dict) -> Dict:
        cve_id = cve['cve']['CVE_data_meta']['ID']
        impact = get_impact_from_cve(cve)
        affected_cpes = get_affected_cpes_from_cve(cve)
        return {'cve_id': cve_id, 'impact': impact, 'vulnerable_cpes': affected_cpes}

    def get_impact_from_cve(cve: Dict) -> Dict:
        result = {'base_score': None, 'severity': None, 'exploitabilityScore': None, 'impactScore': None}
        if not cve['impact']:
            pass
        elif 'baseMetricV3' in cve['impact']:
            result['base_score'] = cve['impact']['baseMetricV3']['cvssV3']['baseScore']
            result['severity'] = cve['impact']['baseMetricV3']['cvssV3']['baseSeverity']
            result['exploitabilityScore'] = cve['impact']['baseMetricV3']['exploitabilityScore']
            result['impactScore'] = cve['impact']['baseMetricV3']['impactScore']
        elif 'baseMetricV2' in cve['impact']:
            result['base_score'] = cve['impact']['baseMetricV2']['cvssV2']['baseScore']
            result['severity'] = cve['impact']['baseMetricV2']['severity']
            result['exploitabilityScore'] = cve['impact']['baseMetricV2']['exploitabilityScore']
            result['impactScore'] = cve['impact']['baseMetricV2']['impactScore']
        return result
    
    def get_affected_cpes_from_cve(cve: Dict) -> List[str]:
        affected_cpes = []
        for node in cve['configurations']['nodes']:
            affected_cpes.extend(get_affected_cpes_from_node(node))
        return affected_cpes

    def get_affected_cpes_from_node(node: Dict) -> List[str]:
        cpe_uris = []
        if 'children' in node:
            for child in node['children']:
                cpe_uris += get_affected_cpes_from_node(child)
        if 'cpe_match' in node:
            lst = node['cpe_match']
            for x in lst:
                if x['vulnerable']:
                    cpe_uris.append(x['cpe23Uri'])
        return cpe_uris


    json_files = glob.glob(cve_dir + '/*.json')
    print(f'Identified {len(json_files)} CVE files. Extracting relevant data and merging them into {output_path}', flush=True)
    
    all_cve_data = []
    for filepath in tqdm(json_files):
        all_cve_data.extend(get_relevant_info_from_file(Path(filepath)))

    with open(output_path, 'w') as handle:
        json.dump(all_cve_data, handle, indent=4)


def get_cpe_uri_to_title_dict(input_xml_filepath: str, output_filepath: str):
    print(f'Extracting dictionary cpe_uri:cpe_title from {input_xml_filepath} to {output_filepath}')
    root = ET.parse(input_xml_filepath).getroot()
    dct = {}
    for cpe_item in root.findall('{http://cpe.mitre.org/dictionary/2.0}cpe-item'):
        title = cpe_item.find('{http://cpe.mitre.org/dictionary/2.0}title').text
        cpe_uri = cpe_item.find('{http://scap.nist.gov/schema/cpe-extension/2.3}cpe23-item').attrib['name']
        dct[cpe_uri] = title
    with open(output_filepath, 'w') as handle:
        json.dump(dct, handle, indent=4)

## Preprocessing realization and path specification

Filepaths for rest of this notebook are specified here. Also, the realized three functions will:
    
1. Download all CVE datafiles
2. Extract relevant CVE information from all files and merge it into single file
3. Create a dictionary of `cpe_uri: cpe title`, will come handly later

In [5]:
CVE_FOLDER_PATH = '/Users/adam/phd/projects/certificates/cpe_matching/new/cves'
CVE_MERGED_FILEPATH = '/Users/adam/phd/projects/certificates/cpe_matching/new/cve_data.json'

CPE_DICTIONARY_PATH = '/Users/adam/phd/projects/certificates/cpe_matching/new/cpe_dictionary.json'
CPE_XML_PATH = '/Users/adam/phd/projects/certificates/cpe_matching/official-cpe-dictionary_v2.3.xml'

PETR_ONE_TO_ONE_MATCH_JSON = '/Users/adam/phd/projects/certificates/cpe_matching/new/certs_to_cpe_single_match.json'
CERTIFICATE_DATASET_CSV = '/Users/adam/phd/projects/certificates/cpe_matching/new/cc_full_dataset.csv'

# download_cve_data(CVE_FOLDER_PATH)
# parse_all_cves(CVE_FOLDER_PATH, CVE_MERGED_FILEPATH)
# get_cpe_uri_to_title_dict(CPE_XML_PATH, CPE_DICTIONARY_PATH)

## Main functions

### CPE dictionary building

In [6]:
def get_cpe_vendor(cpe_record):
    vendor = cpe_record.split(':')[3]
    return ' '.join(vendor.split('_'))

def get_cpe_product(cpe_record):
    return ' '.join(cpe_record.split(':')[4].split('_'))

def get_cpe_version(cpe_record):
    return cpe_record.split(':')[5]

with open(PETR_ONE_TO_ONE_MATCH_JSON, 'r') as handle:
    petrs_matches = json.load(handle)
petrs_matches = {x.split('.pdf')[0]:y for x,y in petrs_matches.items()}

with open(CPE_DICTIONARY_PATH, 'r') as handle:
    cpe_data = json.load(handle)

cpe_triplets = [(get_cpe_vendor(x), get_cpe_product(x), get_cpe_version(x)) for x in cpe_data.keys()]
cpe_uri_to_triplet = {x: (get_cpe_vendor(x), get_cpe_product(x), get_cpe_version(x)) for x in cpe_data.keys()}
cpe_triplet_to_uri = {(get_cpe_vendor(x), get_cpe_product(x), get_cpe_version(x)): x for x in cpe_data.keys()}
cpe_vendor_dict = {x: [] for x in [x[0] for x in cpe_triplets]}
cpe_vendor_to_version_dict = {x: [] for x in [x[0] for x in cpe_triplets]}
cpe_full_dict = {x: [] for x in [(x[0], x[2]) for x in cpe_triplets]}

for vendor, product, version in cpe_triplets:
    cpe_vendor_dict[vendor].append((vendor, product, version))
    cpe_vendor_to_version_dict[vendor].append(version)
    cpe_full_dict[(vendor, version)].append(product)
    
with open(CVE_MERGED_FILEPATH, 'r') as handle:
    cve_dataset = json.load(handle)
vuln_score_mapping = {x['cve_id']: x['impact']['base_score'] for x in cve_dataset}

def get_cve_ids_for_cpe_uri(cpe_uri):
    if not isinstance(cpe_uri, str):
        return None
    if not (ids := [cve['cve_id'] for cve in cve_dataset if cpe_uri in cve['vulnerable_cpes']]):
        return None
    else:
        return ids

## Actual functions for CPE<->Certificate matching

In [7]:
def parse_cert_version(crt_name):
    at_least_something = r'(\b(\d)+\b)'
    just_numbers = r'(\d{1,5})(\.\d{1,5})'
    
    without_version = r'(' + just_numbers + r'+)'
    long_version = r'(' + r'(\bversion)\s*' + just_numbers + r'+)'
    short_version = r'(' + r'\bv\s*' + just_numbers + r'+)'
    full_regex_string = r'|'.join([without_version, short_version, long_version])
    normalizer = r'(\d+\.*)+'

    matched_strings = set([max(x, key=len) for x in re.findall(full_regex_string, crt_name, re.IGNORECASE)])
    if not matched_strings:
        matched_strings = set([max(x, key=len) for x in re.findall(at_least_something, crt_name, re.IGNORECASE)])

    if matched_strings:
        return [re.search(normalizer, x).group() for x in matched_strings]
    else:
        return ['-']
    


def map_petrs_match(report_link):
    for x in petrs_matches.keys():
        if x.replace(' ', '%20') in report_link:
            base_string = 'hotfix:' + petrs_matches[x]
            return get_cpe_vendor(base_string), get_cpe_product(base_string), get_cpe_version(base_string)
    return None

def get_matching_vendors(vendor_name: str) -> Optional[List[str]]:
    result = set()
    if not isinstance(vendor_name, str):
        return None
    lower = vendor_name.lower()
    if ' / ' in vendor_name:
        chain = [get_matching_vendors(x) for x in vendor_name.split(' / ')]
        chain = [x for x in chain if x]
        return list(set(itertools.chain(*chain)))
    if lower in cpe_vendor_dict.keys():
        result.add(lower)
    if ' ' in lower and (y := lower.split(' ')[0]) in cpe_vendor_dict.keys():
        result.add(y)
    if ',' in lower and (y := lower.split(',')[0]) in cpe_vendor_dict.keys():
        result.add(y)
    if not result:
        return None
    return list(result)

def get_matching_versions(cert_versions: List[str], vendor_candidates: List[str]):
    just_numbers = r'(\d{1,5})(\.\d{1,5})'
    matching_versions = set()
    for v in vendor_candidates:
        for c in cert_versions:
            if (c.startswith(v) and re.search(just_numbers, v)) or v.startswith(c):
                matching_versions.add(v)
    return list(matching_versions)

def get_best_match(cert_name: str, list_of_pairs: List[Tuple[str, str]]):
    def sanitize_matched_string(string):
        string = string.replace('®', '').replace('™', '').lower()
        return replace_non_letter_non_numbers_with_space.sub(' ', string)

    best_match = 0
    best_candidate = (None, None, None)
    if not list_of_pairs:
        return best_match, best_candidate

    for vendor, version in list_of_pairs:
        for candidate in cpe_full_dict[(vendor, version)]:
            sanitized_cert_name = sanitize_matched_string(cert_name)
            sanitized_candidate = sanitize_matched_string(candidate)
            potential = max(fuzz.token_set_ratio(sanitized_cert_name, sanitized_candidate), fuzz.partial_ratio(sanitized_cert_name, sanitized_candidate))
            if (potential - best_match) > 1 or \
               (best_candidate[1] and len(best_candidate[1]) > 10 and abs(potential - best_match) < 1 and best_candidate[2] and len(version) < len(best_candidate[2])) or \
               (abs(potential - best_match) < 1 and best_candidate[0] and len(candidate) > len(best_candidate[1])):
                    best_match = potential
                    best_candidate = vendor, candidate, version
    return best_match, best_candidate


def match_cpe(vendor_name: str, cert_name: str, versions: List[str]):
    matching_vendors = get_matching_vendors(vendor_name)
    matching_versions = []
    if not matching_vendors:
        return None, None

    all_candidates = []

    for v in matching_vendors:
        matching_versions.append(get_matching_versions(versions, cpe_vendor_to_version_dict[v]))

    for vendor, versions in zip(matching_vendors, matching_versions):
        all_candidates.extend((vendor, v) for v in versions)

    best_match, best_candidate = get_best_match(cert_name, all_candidates)

    # If we didn't get anything meaningful, try to relax the version and return only if long match and extra certain
    if best_match < 60:
        alt_candidates = [(v, '-') for v in matching_vendors if '-' in cpe_vendor_to_version_dict[v]]
        alt_best_match, alt_best_candidate = get_best_match(cert_name, alt_candidates)
        if alt_best_candidate[1] and len(alt_best_candidate[1]) > 5 and alt_best_match > 70:
            return alt_best_match, alt_best_candidate
    
    return best_match, best_candidate

In [9]:
sec_level_dict = {'EAL1': 0, 'EAL1+': 1, 'EAL2': 2, 'EAL2+': 3, 'EAL3': 4, 'EAL3+': 5, 'EAL4': 6, 'EAL4+': 7, 'EAL5': 8, 'EAL5+': 9, 'EAL6': 10, 'EAL6+': 11, 'EAL7': 12, 'EAL7+': 13}

print(f'Loading and preprocessing the dataframe...')
df = pd.read_csv(CERTIFICATE_DATASET_CSV, sep=';')
df = df.set_index('dgst')

df.security_level = df.security_level.map(ast.literal_eval) # Since we have it in string representation, not needed when deserializing

df['max_security_level'] = df.security_level.map(lambda x: max([sec_level_dict.get(y, -1) for y in x]) if x else -1)

df['version'] = df['name'].map(parse_cert_version)

df.not_valid_before = df.not_valid_before.apply(pd.to_datetime)
df.not_valid_after = df.not_valid_after.apply(pd.to_datetime)

print(f'Doing CPE matching...')
df['petr_match'] = df.report_link.map(map_petrs_match)
df['adam_match'] = df.apply(lambda x: match_cpe(x['manufacturer'], x['name'], x['version']), axis=1)

print(f'Doing match postprocessing...')
df['match_score'] = df.adam_match.apply(lambda x: x[0])
df['adam_match'] = df.adam_match.apply(lambda x: x[1])

df['has_long_cpe_match'] = df.adam_match.apply(lambda x: len(x[1]) > 5 if x and x[1] else False)
df['matched_cpe_uri'] = df.adam_match.map(cpe_triplet_to_uri)

df_full = df.copy()

# # Filter only to relevant pieces

df = df.loc[df.has_long_cpe_match == True]
df = df.loc[df.match_score > 80]

print(f'Retrieving CVEs for cpe-rich certificates', flush=True)
df['related_cves'] = df.matched_cpe_uri.progress_map(get_cve_ids_for_cpe_uri)
df['n_related_cves'] = df.related_cves.apply(lambda x: len(x) if x else 0)

Doing CPE matching...
Doing match postprocessing...


  1%|          | 4/598 [00:00<00:21, 27.62it/s]

Retrieving CVEs for cpe-rich certificates


 56%|█████▌    | 334/598 [00:15<00:12, 21.33it/s]

In [ ]:
df.shape

In [ ]:
df_cves = df.explode('related_cves')
df_cves = df_cves.reset_index()
df_cves['cve_score'] = df_cves.related_cves.map(vuln_score_mapping)
df_cves = df_cves.loc[df_cves.n_related_cves < 100]
df_cves = df_cves.loc[df_cves.max_security_level > -1]

### Compare distribution of years in All vs CPE-rich certificates

In [ ]:
years_matched = df.not_valid_before.dt.year.value_counts().sort_index().rename('# all certificates')
years_all = df_full.not_valid_before.dt.year.value_counts().sort_index().rename('# CPE-rich certificates')
years_merged = pd.concat([years_all, years_matched], axis=1)
years_merged = years_merged.fillna(0)
years_merged = years_merged.div(years_merged.sum(axis=0), axis=1)
ax = years_merged.plot(title='Proportion of certificates not-valid-before given year')
fig = ax.get_figure()
fig.savefig('/Users/adam/Downloads/n_certs.png', dpi=300)

## Compare distribution of categories between all vs CPE-rich certificates

In [ ]:
categories_filtered = df.category.value_counts().sort_index().rename('Category distribution CPE-rich')
categories_all = df_full.category.value_counts().sort_index().rename('Category distribution all')
categories_merged = pd.concat([categories_filtered, categories_all], axis=1)
categories_merged = categories_merged.drop('ICs, Smart Cards and Smart Card-Related Devices and Systems')
categories_merged = categories_merged.div(categories_merged.sum(axis=0), axis=1)
ax = categories_merged.plot.bar(title='Categories (without smartcards) comparison between CPE-rich and all certificates')
fig = ax.get_figure()
fig.savefig('/Users/adam/Downloads/categories.png', dpi=300)

## Compare distribution of EAL levels between all vs CPE-rich certificates

In [ ]:
levels_filtered = df.max_security_level.value_counts().sort_index().rename('Sec. level distribution CPE-rich')
levels_all = df_full.max_security_level.value_counts().sort_index().rename('Sec. level distribution all')
levels_merged = pd.concat([levels_filtered, levels_all],  axis=1)
levels_merged = levels_merged.fillna(0)
levels_merged = levels_merged.div(levels_merged.sum(axis=0), axis=1)
ax = levels_merged.plot.bar(title='Security levels in CPE-rich certificate vs all certificates')
fig = ax.get_figure()
fig.savefig('/Users/adam/Downloads/security_levels.png', dpi=300)

In [ ]:
df_cves.plot.hexbin(x='max_security_level',
                    y='cve_score',
                    gridsize=20,
                    colormap='viridis')

In [ ]:
fig, ax = plt.subplots()
ax = df_cves.plot.scatter('max_security_level', 'cve_score', c='n_related_cves', colormap='viridis',
                         s = 40,
                         title='CVE score vs. security level of affected certificate. Color = number of CVEs related to certificate',
                         xlabel='Security level, EAL1=0, EAL7+=13',
                         ylabel='CVE severity score 1-10',
                         figsize=(12,10), ax=ax)
fig = ax.get_figure()
fig.savefig('/Users/adam/Downloads/scatter_plot.png', dpi=300)

In [ ]:
# Difference between Adam's and Petr's matching
interesting_cols = ['name', 'manufacturer', 'version', 'match_score', 'adam_match', 'petr_match']
df_diff = df_full.loc[(df_full.petr_match.notnull()) & (df_full.petr_match != df_full.adam_match), interesting_cols]
df_diff[interesting_cols]

In [95]:
df_cves.loc[df_cves.cve_score > 9.5]

,dgst,name,status,category,manufacturer,scheme,security_level,not_valid_before,not_valid_after,report_link,...,max_security_level,version,petr_match,adam_match,match_score,has_long_cpe_match,matched_cpe_uri,related_cves,n_related_cves,cve_score
5,1d961fc9a50e0263,IBM Security Access Manager for Enterprise Single Sign-On Version 8.2,active,Access Control Devices and Systems,IBM Corporation,DE,"[ALC_FLR.1, EAL3+]",2014-12-05,NaT,http://commoncriteriaportal.org/files/epfiles/0683a_pdf.pdf,...,5,[8.2],"(ibm, security access manager for enterprise single sign-on, 8.2)","(ibm, security access manager for enterprise single sign-on, 8.2)",100.000000,True,cpe:2.3:a:ibm:security_access_manager_for_enterprise_single_sign-on:8.2:*:*:*:*:*:*:*,CVE-2015-0235,4,10.0
62,409262d15afed629,McAfee Data Loss Prevention 11.0 with ePolicy Orchestrator 5.9.0,active,Data Protection,"McAfee, LLC.",CA,"[ALC_FLR.2, EAL2+]",2018-01-04,2023-01-04,http://commoncriteriaportal.org/files/epfiles/383-4-429%20CR%201.0e.pdf,...,3,"[11.0, 5.9.0]","(mcafee, epolicy orchestrator, 5.9.0)","(mcafee, epolicy orchestrator, 5.9.0)",100.000000,True,cpe:2.3:a:mcafee:epolicy_orchestrator:5.9.0:*:*:*:*:*:*:*,CVE-2017-3936,12,9.8
88,7b1ea55f7e883dbb,MarkLogic Server 8.0-4,active,Databases,MarkLogic Corporation,MY,"[ALC_FLR.3, EAL2+]",2015-12-22,NaT,http://commoncriteriaportal.org/files/epfiles/ISCB-5-RPT-C069-CR-v1.pdf,...,3,[8.0],None,"(marklogic, marklogic, 8.0-6)",100.000000,True,cpe:2.3:a:marklogic:marklogic:8.0-6:*:*:*:*:*:*:*,CVE-2017-2792,8,9.6
920,fcbfeedd213452dd,WorkCentre® 5735/5740/5745/5755/5765/5775/5790 with FIPS 140-2 Compliance over SNMPv3,active,Multi-Function Devices,Xerox Corporation,CA,"[ALC_FLR.3, EAL2+]",2016-07-28,2021-07-28,http://commoncriteriaportal.org/files/epfiles/383-4-370%20CR%20v1.0e.pdf,...,3,"[5790, 5755, 5740, 5765, 2, 5735, 5775, 140, 5745]",None,"(xerox, workcentre, 275)",100.000000,True,cpe:2.3:h:xerox:workcentre:275:*:*:*:*:*:*:*,CVE-2009-1656,1,10.0
922,32584d0c27c2135c,WorkCentre® 7525/7530/7535/7545/7556 with FIPS 140-2 Compliance over SNMPv3,active,Multi-Function Devices,Xerox Corporation,CA,"[ALC_FLR.3, EAL2]",2016-07-25,2021-07-25,http://commoncriteriaportal.org/files/epfiles/383-4-371%20CR%20v1.0e.pdf,...,2,"[7556, 7535, 7545, 2, 7530, 140, 7525]",None,"(xerox, workcentre, 275)",100.000000,True,cpe:2.3:h:xerox:workcentre:275:*:*:*:*:*:*:*,CVE-2009-1656,1,10.0
1035,c348756b08b0637a,F5 Networks BIG-IP® Application Delivery Controller (ADC-AP) version 11.5.1 HF10 (build 10.123.180),active,Network and Network-Related Devices and Systems,"F5 Networks, Inc.",DE,"[ALC_FLR.3, EAL4+]",2018-02-15,2023-02-15,http://commoncriteriaportal.org/files/epfiles/0975a_pdf.pdf,...,7,"[10.123.180, 11.5.1]","(f5, big-ip controller, 1.5.1)","(f5, big-ip link controller, 11.5.1)",87.179487,True,cpe:2.3:a:f5:big-ip_link_controller:11.5.1:*:*:*:*:*:*:*,CVE-2017-6165,35,9.8
1040,c348756b08b0637a,F5 Networks BIG-IP® Application Delivery Controller (ADC-AP) version 11.5.1 HF10 (build 10.123.180),active,Network and Network-Related Devices and Systems,"F5 Networks, Inc.",DE,"[ALC_FLR.3, EAL4+]",2018-02-15,2023-02-15,http://commoncriteriaportal.org/files/epfiles/0975a_pdf.pdf,...,7,"[10.123.180, 11.5.1]","(f5, big-ip controller, 1.5.1)","(f5, big-ip link controller, 11.5.1)",87.179487,True,cpe:2.3:a:f5:big-ip_link_controller:11.5.1:*:*:*:*:*:*:*,CVE-2016-5022,35,9.8
1041,c348756b08b0637a,F5 Networks BIG-IP® Application Delivery Controller (ADC-AP) version 11.5.1 HF10 (build 10.123.180),active,Network and Network-Related Devices and Systems,"F5 Networks, Inc.",DE,"[ALC_FLR.3, EAL4+]",2018-02-15,2023-02-15,http://commoncriteriaportal.org/files/epfiles/0975a_pdf.pdf,...,7,"[10.123.180, 11.5.1]","(f5, big-ip controller, 1.5.1)","(f5, big-ip link controller, 11.5.1)",87.179487,True,cpe:2.3:a:f5:big-ip_link_controller:11.5.1:*:*:*:*:*:*:*,CVE-2016-5700,35,9.8
1208,9675b63656e6b090,Dell EMC™ Avamar® v18.1,active,Other Devices and Systems,Dell EMC,CA,"[ALC_FLR.2, EAL2+]